<h1>CI Final SQL Project</h1>

<h2>You are working at a financial firm and they have taken over a smaller client. <br>
Your job is to convert their customers and records into your format.  <br>
There are 2 type of records corporate and customer <br>
<br></h2>
<br>
<b>NOTE: </b>All financial data can be found at https://finance.yahoo.com<br>
Download data for the following list of securities:<br>
'COF','DIS','SQ','INTC','MSFT','CRM','CMG','NKE','AMD','UA'<br>

** You should download the last 5 years worth of data.<br>

#### 1. Place your project directory variable in the code block below - 5 points
```
project_dir = 'C:\\temp\final_project\\'  # <--- USE YOUR DIRECTORY
# 5 easy points
```

In [ ]:
project_dir = '/Users/yani/Desktop/PY-112-MySQL/Final Project'

<h4>2. Download the historical data (5 years) from finance.yahoo.com - 5 points<br>
print a list of all csv files in your project directory</h4>

In [ ]:
import os 
os.chdir('/Users/yani/Desktop/PY-112-MySQL/Final Project')
files = os.listdir('.')

py_files = [py_file for py_file in os.listdir('.') if py_file.endswith(".csv")]
print(py_files)

<h4>3. Create a database called 'sql_final_project' - 5 points</h4>

In [ ]:
import mysql.connector
from dotenv import dotenv_values

config = dotenv_values(".env")  

mydb = mysql.connector.connect(

  host=config['HOST'],
  user=config['USER'],
  password=config['PASSWORD'],
  database=config['DATABASE'])

mycursor = mydb.cursor()

mycursor.execute('Create Database If Not Exists sql_final_project;')

mydb.commit()
mydb.close()

<h4>4. Print the name of all existing databases - 5 points</h4>

In [ ]:
import mysql.connector
from dotenv import dotenv_values

config = dotenv_values(".env")  

mydb = mysql.connector.connect(
  host=config['HOST'],
  user=config['USER'],
  password=config['PASSWORD'],
  database=config['DATABASE'])

mycursor = mydb.cursor()

mycursor.execute('Show Databases;')
for x in mycursor:
  print(x)

mydb.close()

<h4>5. Create the tables listed below using the proper data types: - 10 points</h4>
- 'security_master' with the following fields: sec_symbol, trade_date, closing_price, volume (discard the rest)<br>
  (THIS IS THE YAHOO FINANCE DATA)<br>
- 'cust_accounts' with the following fields:<br>
  acct_num, trade_type, gender, last_name, first_name, salary, city, state  <br>   
- 'cust_transactions' with the following fields: <br>
  trade_date, sec_symbol, acct_num, trade_type, trade_lot, transaction_id<br>
- 'corp_accounts' with the following fields:<br>
  acct_num, acct_type, acct_name, trading_limit (drop the address and zip info)   <br> 
- 'corp_transactions' with the following fields: <br>
  trade_date, sec_symbol, acct_num, trade_type, trade_lot, transaction_id<br>

In [ ]:
import mysql.connector
from dotenv import dotenv_values

config = dotenv_values(".env")  

mydb = mysql.connector.connect(
  host=config['HOST'],
  user=config['USER'],
  password=config['PASSWORD'],
  database=config['DATABASE'])

mycursor = mydb.cursor()

mycursor.execute('Use sql_final_project;')

# 'security_master' with the following fields: sec_symbol, trade_date, closing_price, volume (discard the rest)

# mycursor.execute('Drop Table If Exists security_master;')
mycursor.execute('''Create Table If Not Exists security_master
(
sec_symbol      varchar(10)  Not Null, 
trade_date      Date         Not Null, 
closing_price   Float        Not Null, 
volume          int          Not Null);
''')

# mycursor.execute('Drop Table if Exists cust_accounts;')
# mycursor.execute('Drop Table if Exists cust_transactions;')

# # - 'cust_transactions' with the following fields: trade_date, sec_symbol, acct_num, trade_type, trade_lot, transaction_id
mycursor.execute('''Create Table If Not Exists cust_transactions
(
acct_num        varchar(100)    Not Null,
trade_date      Date            Not Null,
sec_symbol      varchar(25)     Not Null,
trade_type      varchar(10)     Not Null,
trade_lot       int(10)         Not Null,
transaction_id  int(20)         Not Null,
index(acct_num));
''')



# 'cust_accounts' with the following fields: acct_num, trade_type, gender, last_name, first_name, salary, city, state
mycursor.execute('''Create Table If Not Exists cust_accounts
(
acct_num        varchar(100)  Not Null,  
gender          varchar(1)  Not Null, 
last_name       varchar(100)Not Null,
first_name      varchar(100)    Not Null,
salary          int             Not Null,
city            varchar(50)     Not Null,
state           varchar(2)      Not Null,
Constraint fk_cust_acct_num
Foreign Key (acct_num)
  References cust_transactions(acct_num)
	On Update Cascade
    On Delete Cascade);
''')



# # # - 'corp_accounts' with the following fields: acct_num, acct_type, acct_name, trading_limit (drop the address and zip info)
mycursor.execute('''Create Table If Not Exists corp_accounts
(
acct_num        varchar(100)    Not Null    Primary Key,
acct_type       varchar(10)     Not Null,
acct_name       varchar(150)    Not Null,
trading_limit   Int(20)         Not Null,
city            varchar(100)    Not Null,
state           varchar(2)      Not Null);
''')

# # # - 'corp_transactions' with the following fields: trade_date, sec_symbol, acct_num, trade_type, trade_lot, transaction_id
mycursor.execute('''Create Table if Not Exists corp_transactions
(
acct_num    varchar(100)    Not Null,
trade_date  Date            Not Null, 
sec_symbol      varchar(10)     Not Null,
trade_type      varchar(10)     Not Null,
trade_lot       int(10)         Not Null,
transaction_id  int(20)         Not Null,
Constraint fk_corp_acct_num
Foreign Key (acct_num)
  References corp_accounts(acct_num)
    On update Cascade
    On delete Cascade);
''')


mydb.commit()
mydb.close()

<h4>6. Populate the security_master table - 10 points</h4> 

When testing this out, tack on a limit 25 or so , doing a select * on the table with no limit is very memory intensive!


In [ ]:
# 'security_master' with the following fields: sec_symbol, trade_date, closing_price, volume (discard the rest)
import csv
import os

os.chdir(r'/Users/yani/Desktop/PY-112-MySQL/Final Project')

files = os.listdir('.')

py_files = [py_file for py_file in os.listdir('.') if py_file.endswith(".csv")]
# # print(py_files)

securities_files = [x for x in py_files if len(x) < 9]
# print(securities_files)

security_data = []
def extract_security_data(li = []):
    for x in li:
        sec_symbol = (x.split('.'))[0]
        with open(x, newline='') as csv_file:
            data = csv.DictReader(csv_file)
            for row in data:
                security_data.append((sec_symbol, row['Date'], row['Close'], row['Volume']))
    

extract_security_data(securities_files)
# print(security_data[:5])

''' Pupulate Table security_master'''

import mysql.connector
from dotenv import dotenv_values

config = dotenv_values(".env")  

mydb = mysql.connector.connect(
  host=config['HOST'],
  user=config['USER'],
  password=config['PASSWORD'],
  database=config['DATABASE'])

mycursor = mydb.cursor()

mycursor.execute('Use sql_final_project;')

mycursor.executemany('''Insert Into security_master (sec_symbol, trade_date, closing_price, volume) Values (%s, %s, %s, %s);''', security_data)


mydb.commit()
mycursor.execute('''Select * From security_master Limit 25;''')

for x in mycursor:
  print(x)


mydb.close()



<h4>7. Calculate the highest closing price and lowest closing price for every security after 3/17/2020</h4>
- 5 points

In [ ]:
import mysql.connector
from dotenv import dotenv_values

config = dotenv_values(".env")  

mydb = mysql.connector.connect(
  host=config['HOST'],
  user=config['USER'],
  password=config['PASSWORD'],
  database=config['DATABASE'])

mycursor = mydb.cursor()

mycursor.execute('Use sql_final_project;')
# Highest Closing Price
mycursor.execute('''Select * From security_master
Where trade_date > "2020-03-17"
Order By closing_price Desc
Limit 1;''')

highest_closing_price = mycursor.fetchone()
print(highest_closing_price)

# Lowest Closing Price
mycursor.execute('''Select * From security_master
Where trade_date > "2020-03-17"
Order By closing_price 
Limit 1;''')

lowest_closing_price = mycursor.fetchone()
print(lowest_closing_price)

mydb.close()



<h4>8. Import and clean the following files</h4>
1. corp_accounts.csv<br>
2. corp_transactions<br>
3. cust_accounts<br>
4. cust_transactions<br>
5 points

i.e. 

```
CO-37971|CORP|rahway racing|5363 Broadway Avenue|greenwich| CT   | 30437-0811|$93900000
```
- get rid of trailing white spaces in corp_accounts.csv.

```
CU-99956733,RMS,M,CHAVEZ, alexander,$150755,saratoga%springs,NY,
```
- get rid of % and trailing comma in the names of places in cust_accounts.csv


In [ ]:
'''Cleaning Data and Importing into cust_transactions'''
import csv
import os
import string

os.chdir(r'/Users/yani/Desktop/PY-112-MySQL/Final Project')

import mysql.connector
from dotenv import dotenv_values

config = dotenv_values(".env") 

mydb = mysql.connector.connect(
  host=config['HOST'],
  user=config['USER'],
  password=config['PASSWORD'],
  database=config['DATABASE'])

mycursor = mydb.cursor()

# 'cust_transactions': acct_num, trade_date, sec_symbol, trade_type, trade_lot, transaction_id
with open('cust_transactions.csv', newline = '') as cust_transactions:
    data2 = csv.DictReader(cust_transactions)
    cust_transactions_li = []
    symbols = string.printable[(string.printable.index('!')):].replace('-','')
    for row in data2:
        # Removes symbols (except '-') found in any of the dict values:
        for x,y in row.items():
            for i in y:
                if i in symbols:
                    # print(row)
                    row[x] = str(y).replace(i, ' ')

        row['acct_num']=row['acct_num'].strip().upper()
        row['trade_date']= row['trade_date'].strip()
        row['sec_symbol'] = row['sec_symbol'].strip().upper()
        row['trade_type'] = row['trade_type'].upper().strip()
        row['trade_lot'] = row['trade_lot'].strip()
        row['transaction_id'] = row['transaction_id'].strip()
    

        cust_transactions_tuple = (row['acct_num'], row['trade_date'], row['sec_symbol'], row['trade_type'], row['trade_lot'], row['transaction_id'])

        query = f'''Insert Into cust_transactions (acct_num, trade_date, sec_symbol, trade_type, trade_lot, transaction_id) Values {cust_transactions_tuple}'''
        
        mycursor.execute(query)
        
mydb.commit()

mycursor.execute('''Select * From cust_transactions Limit 15;''')
for x in mycursor:
    print(x)

mydb.close()

In [ ]:
'''Cleaning Data and importing into cust_accounts'''
import csv
import os
import string
import pprint

os.chdir(r'/Users/yani/Desktop/PY-112-MySQL/Final Project')

import mysql.connector
from dotenv import dotenv_values

config = dotenv_values(".env")  

mydb = mysql.connector.connect(
  host=config['HOST'],
  user=config['USER'],
  password=config['PASSWORD'],

  database=config['DATABASE'])

mycursor = mydb.cursor()
# 'cust_accounts': acct_num, trade_type (will come from the cust_transactions table), gender, last_name, first_name, salary, city, state

with open('cust_accounts.csv', newline = '') as cust_accounts:
    data1 = csv.DictReader(cust_accounts)
    
    # Get string of symbols from ascii characters:
    symbols = string.printable[(string.printable.index('!')):].replace('-','')
    for row in data1:
        row.pop('') #Removes the dict key, value pair for the random keyname = None.  
        # Removes symbols (except '-') found in any of the dict values:
        for x,y in row.items():
            for i in y:
                if i in symbols:
                    # print(row)
                    row[x] = str(y).replace(i, ' ')
        row['acct_num'] = row['acct_num'].strip().upper()
        # print(row['acct_num'])
        row['gender'] = str(row['gender'].strip())
        row['last_name'] = row['last_name'].strip().capitalize()
        row['first_name'] = row['first_name'].strip().capitalize()
        row['salary'] = row['salary'].replace("$", "").strip()
        row['city'] = row['city'].strip().title()
        row['state'] = row['state'].strip().upper()

        # Could not insert data using executemany.
        cust_accounts_tuple = (row['acct_num'], row['gender'], row['last_name'], row['first_name'], row['salary'], row['city'], row['state'])
        query = f'''Insert Into cust_accounts (acct_num,gender,last_name,first_name,salary,city,state) Values {cust_accounts_tuple}'''
        # print(query)

        mycursor.execute(query)
        
mydb.commit()

mycursor.execute('''Select * From cust_accounts Limit 15;''')
for x in mycursor:
    print(x)

mydb.close()


In [ ]:
'''Cleaning Data and importing into corp_accounts'''
import csv
import os
import string

os.chdir(r'/Users/yani/Desktop/PY-112-MySQL/Final Project')

import mysql.connector
from dotenv import dotenv_values

config = dotenv_values(".env")  

mydb = mysql.connector.connect(
  host=config['HOST'],
  user=config['USER'],
  password=config['PASSWORD'],
  database=config['DATABASE'])

mycursor = mydb.cursor()

# - 'corp_accounts': acct_num, acct_type, acct_name, trading_limit, city, state. (drop the address and zip info)
with open('corp_accounts.csv', newline = '') as corp_accounts:
    data3 = csv.DictReader(corp_accounts, delimiter = '|')
    
    symbols = string.printable[(string.printable.index('!')):].replace('-','')
    for row in data3:
      # Removes symbols (except '-') found in any of the dict values:
      for x,y in row.items():
        row[x] = y.strip()
        for i in y:
          if i in symbols:
            row[x] = str(y).replace(i, ' ')
      row['acct_num'] = row['acct_num'].strip().upper()
      row['acct_type'] = row['acct_type'].upper().strip()
      row['acct_name'] = row['acct_name'].title().strip()
      row['trading_limit'] = row['trading_limit'].strip()
      row['city'] = row['city'].strip().capitalize()
      row['state'] = row['state'].strip().upper()

      corp_accounts_tuple = (row['acct_num'], row['acct_type'], row['acct_name'], row['trading_limit'], row['city'], row['state'])

      query = f'''Insert Into corp_accounts (acct_num, acct_type, acct_name, trading_limit, city, state) Values {corp_accounts_tuple}'''
      mycursor.execute(query)

mydb.commit()
mydb.close()

In [ ]:
'''Cleaning Data and importing into corp_transactions'''
import csv
import os
import string

os.chdir(r'/Users/yani/Desktop/PY-112-MySQL/Final Project')

# - 'corp_transactions': acct_num, trade_date, sec_symbol, trade_type, trade_lot, transaction_id
with open('corp_transactions.csv', newline = '') as corp_transactions:
    data4 = csv.DictReader(corp_transactions)
    corp_transactions_li = []
    symbols = string.printable[(string.printable.index('!')):].replace('-','')
    for row in data4:
      # Removes symbols (except '-') found in any of the dict values:
      for x,y in row.items():
            for i in y:
                if i in symbols:
                    row[x] = str(y).replace(i, ' ')
      
      corp_transactions_li.append((row['acct_num'].strip().upper(), row['trade_date'].strip(), row['sec_symbol'].upper().strip(), row['trade_type'].upper().strip(), row['trade_lot'].strip(), row['transaction_id'].strip()))

# print(corp_transactions_li[:2])


import mysql.connector
from dotenv import dotenv_values

config = dotenv_values(".env")  

mydb = mysql.connector.connect(
  host=config['HOST'],
  user=config['USER'],
  password=config['PASSWORD'],
  database=config['DATABASE'])

mycursor = mydb.cursor()

mycursor.executemany('''Insert Into corp_transactions (acct_num, trade_date, sec_symbol, trade_type, trade_lot, transaction_id) Values (%s, %s, %s, %s, %s, %s);''', corp_transactions_li)


mydb.commit()
mydb.close()

<h4>9. Load the data from the csv files into the tables you created and cleaned from step 8</h4>
- 5 points

In [ ]:
''' I loaded, cleaned and uploaded the data for each table in their own individual cells above. I did this due to not being able to add data using 'executemany' for some of the tables.'''

<h4>10. Calculate the Profit and Loss (PnL) for every trade in the customer accounts 
and corp accounts (2 SQL queries) by security by Buy/Sell</h4>
10 points

In [ ]:
import mysql.connector
from dotenv import dotenv_values
import os
import pprint

os.chdir(r'/Users/yani/Desktop/PY-112-MySQL/Final Project')

config = dotenv_values(".env")  

mydb = mysql.connector.connect(
  host=config['HOST'],
  user=config['USER'],
  password=config['PASSWORD'],
  database=config['DATABASE'])

mycursor = mydb.cursor()

#PnL for Corp Accounts
mycursor.execute("""SELECT a.acct_num As 'Account Number', a.transaction_id As 'Transaction ID', a.sec_symbol As 'Security', a.trade_lot as 'Shares Purchased', c.closing_price As 'Purchase Price', b.trade_lot As 'Shares Sold',d.closing_price As 'Sale Price',
( a.trade_lot * (d.closing_price-c.closing_price  )) as PnL
FROM corp_transactions a 
LEFT JOIN corp_transactions b
ON a.sec_symbol = b.sec_symbol and a.transaction_id = b.transaction_id
and b.trade_type = 'SELL'
LEFT JOIN security_master c
ON a.sec_symbol = c.sec_symbol and a.trade_date = c.trade_date
LEFT JOIN security_master d
ON b.sec_symbol = d.sec_symbol and b.trade_date = d.trade_date
WHERE a.trade_type = 'BUY'
LIMIT 15;""")

pnl_corp = mycursor.fetchall()
pprint.pp(f'PnL for corporate accounts:{pnl_corp}')

# Pnl for Customer Accounts
mycursor.execute("""SELECT a.acct_num As 'Account Number', a.transaction_id As 'Transaction ID', a.sec_symbol As 'Security', a.trade_lot as 'Shares Purchased', c.closing_price As 'Purchase Price', b.trade_lot As 'Shares Sold',d.closing_price As 'Sale Price',
( a.trade_lot * (d.closing_price-c.closing_price  )) as PnL
FROM cust_transactions a 
LEFT JOIN cust_transactions b
ON a.sec_symbol = b.sec_symbol and a.transaction_id = b.transaction_id
and b.trade_type = 'SELL'
LEFT JOIN security_master c
ON a.sec_symbol = c.sec_symbol and a.trade_date = c.trade_date
LEFT JOIN security_master d
ON b.sec_symbol = d.sec_symbol and b.trade_date = d.trade_date
WHERE a.trade_type = 'BUY' 
LIMIT 15;""")

pnl_cust = mycursor.fetchall()
pprint.pp(f'PnL for customer accounts:{pnl_cust}')

mydb.close()

In [ ]:
# your code here

<h4>11. Write a query to report by account number the total number of buys, sells, and shares traded per security
    (For both customer and corporate accounts - 2 SQL queries)<br>
</h4> 
5 points + 5 Extra points for combining the 2 queries in one output

In [ ]:
import mysql.connector
from dotenv import dotenv_values

config = dotenv_values(".env")  

mydb = mysql.connector.connect(
  host=config['HOST'],
  user=config['USER'],
  password=config['PASSWORD'],
  database=config['DATABASE'])

mycursor = mydb.cursor()

# Corporate Accounts:


mycursor.execute('''select acct_num, sec_symbol, count(trade_type) As "# of Buys", sum(trade_lot) As "Shares Traded"
from corp_transactions
where trade_type = "BUY"
group by sec_symbol, acct_num
order by acct_num;''')

corp_buys = mycursor.fetchall()[0]
print(corp_buys)

mycursor.execute("""select acct_num, sec_symbol, count(trade_type) As '# of Sells', sum(trade_lot) As 'Shares Traded'
from corp_transactions
where trade_type = 'SELL'
group by sec_symbol, acct_num
order by acct_num;""")

corp_sells = mycursor.fetchall()[0]
print(corp_sells)

# Customer Accounts:

mycursor.execute("""select acct_num, sec_symbol, count(trade_type) As '# of Buys', sum(trade_lot) As 'Shares Traded'
from cust_transactions
where trade_type = 'BUY'
group by sec_symbol, acct_num
order by acct_num;""")

cust_buys = mycursor.fetchall()[0]
print(cust_buys)

mycursor.execute("""select acct_num, sec_symbol, count(trade_type) As '# of Sells', sum(trade_lot) As 'Shares Traded'
from cust_transactions
where trade_type = 'SELL'
group by sec_symbol, acct_num
order by acct_num;""")

cust_sells = mycursor.fetchall()[0]
print(cust_sells)

mydb.close()

<h4>12.  Calculate the 3 biggest losses per account per security</h4>
5 points

In [ ]:

import mysql.connector
from dotenv import dotenv_values
import os
import pprint

os.chdir(r'/Users/yani/Desktop/PY-112-MySQL/Final Project')

config = dotenv_values(".env")  

mydb = mysql.connector.connect(
  host=config['HOST'],
  user=config['USER'],
  password=config['PASSWORD'],
  database=config['DATABASE'])

mycursor = mydb.cursor()

#Biggest Losses in Corporate Accounts
mycursor.execute("""SELECT a.acct_num As 'Account Number', a.transaction_id As 'Transaction ID', a.sec_symbol As 'Security', a.trade_lot as 'Shares Purchased', c.closing_price As 'Purchase Price', b.trade_lot As 'Shares Sold',d.closing_price As 'Sale Price',
( a.trade_lot * (d.closing_price-c.closing_price  )) as PnL
FROM corp_transactions a 
LEFT JOIN corp_transactions b
ON a.sec_symbol = b.sec_symbol and a.transaction_id = b.transaction_id
and b.trade_type = 'SELL'
LEFT JOIN security_master c
ON a.sec_symbol = c.sec_symbol and a.trade_date = c.trade_date
LEFT JOIN security_master d
ON b.sec_symbol = d.sec_symbol and b.trade_date = d.trade_date
WHERE a.trade_type = 'BUY' and ( a.trade_lot * (d.closing_price-c.closing_price  )) < 0
order by pnl
LIMIT 3;
""")

biggest_corp_losses = mycursor.fetchall()
pprint.pp(biggest_corp_losses)


#Biggest Losses in Customer Accounts
mycursor.execute("""SELECT a.acct_num As 'Account Number', a.transaction_id As 'Transaction ID', a.sec_symbol As 'Security', a.trade_lot as 'Shares Purchased', c.closing_price As 'Purchase Price', b.trade_lot As 'Shares Sold',d.closing_price As 'Sale Price',
( a.trade_lot * (d.closing_price-c.closing_price  )) as PnL
FROM cust_transactions a 
LEFT JOIN cust_transactions b
ON a.sec_symbol = b.sec_symbol and a.transaction_id = b.transaction_id
and b.trade_type = 'SELL'
LEFT JOIN security_master c
ON a.sec_symbol = c.sec_symbol and a.trade_date = c.trade_date
LEFT JOIN security_master d
ON b.sec_symbol = d.sec_symbol and b.trade_date = d.trade_date
WHERE a.trade_type = 'BUY' and ( a.trade_lot * (d.closing_price-c.closing_price  )) < 0
order by pnl
LIMIT 3;
""")

biggest_cust_losses = mycursor.fetchall()
pprint.pp(biggest_cust_losses)

mydb.close()



<h4>13. MSFT was on the restricted list for the corporate client Ether Trading from 1/1/2017 through 12/31/2017<br>
Please list all trades for this client during this period and classify the restricted trades as 'ILLEGAL'<br>
Part 2 is to fine them $10,000 for every trade.  How much in fines do they owe?</h4>
10 points

In [ ]:
import mysql.connector
from dotenv import dotenv_values

config = dotenv_values(".env")  

mydb = mysql.connector.connect(
  host=config['HOST'],
  user=config['USER'],
  password=config['PASSWORD'],
  database=config['DATABASE'])

mycursor = mydb.cursor()

# List of Illegal Transactions in given date range
mycursor.execute("""Select ca.acct_name, ct.acct_num, ct.trade_date, ct.sec_symbol, ct.trade_type As 'Illegal Trades'
From corp_transactions ct
Inner Join corp_accounts ca
On ct.acct_num = ca.acct_num
Where ca.acct_name = 'ether trading' and ct.sec_symbol = 'MSFT' and trade_date between '2017-01-01' and '2017-12-31'""")

illegal_transactions = mycursor.fetchall()
pprint.pp(illegal_transactions)

# Total Fine Owed
mycursor.execute("""Select (count(*) * 10000) As 'Total Fine' 
from corp_transactions ct
Inner Join corp_accounts ca
On ct.acct_num = ca.acct_num
Where ca.acct_name = 'ether trading' and ct.sec_symbol = 'MSFT' and trade_date between '2017-01-01' and '2017-12-31';""")

total_fine = mycursor.fetchall()
print(f'The total fine owed is ${total_fine}')

mydb.close()


<h4>14. The SEC wants you to send them a report of all trades done by the following customer accounts:<br>
CU-94677950<br>
CU-26735586<br>
CU-04842147<br>
CU-52277499<br>
For the calendar year 2016 and 2017<br>
The report should be written to disk and contain:<br>
acct_num, last name, first name, security, trade date, lot size, current price on that day<br>
The report should be in date order by security symbol<br>
File name 'Suspicious_activity_YYYYMMDD.csv'<br>
10 points

In [ ]:
''' I attempted to run the query below but kept getting the following error:
"Error Code: 1290. The MySQL server is running with the --secure-file-priv option so it cannot execute this statement"
I could not change the value of the variable mentioned. Therefore, I simply exported the result set as a csv file'''

import mysql.connector
from dotenv import dotenv_values
import os
import pprint

os.chdir(r'/Users/yani/Desktop/PY-112-MySQL/Final Project')

config = dotenv_values(".env")  

mydb = mysql.connector.connect(
  host=config['HOST'],
  user=config['USER'],
  password=config['PASSWORD'],
  database=config['DATABASE'])

mycursor = mydb.cursor()

mycursor.execute("""Select ca.acct_num, ca.last_name, ca.first_name, ct.sec_symbol, ct.trade_date, ct.trade_type, ct.trade_lot, ct.transaction_id, sm.closing_price
from cust_accounts ca
Inner Join cust_transactions ct
on ca.acct_num = ct.acct_num
Inner Join security_master sm
on ct.trade_date = sm.trade_date and ct.sec_symbol = sm.sec_symbol
Where (ca.acct_num = 'CU-94677950' or ca.acct_num = 'CU-26735586' or ca.acct_num = 'CU-04842147' or ca.acct_num = 'CU-52277499') 
and (ct.trade_date between '2016-01-01' and '2017-12-31')
order by ct.sec_symbol and ct.trade_date
Into Outfile '/Users/yani/Desktop/PY-112-MySQL/Final Project/Suspicious_activity_20210720.csv'
Fields Enclosed By ';'
Escaped By '"'
Lines Terminated By '\r\n';""")

mydb.close()


<h4>15. Label each trade on customer accounts as either 'STCG' for Short Term Capital Gains if <br>
the trading length was under 365 days or 'LTCG' Long Term Capital Gains for each security held over 365 days<br>
In customer order by trade date<br></h4>
5 points

In [ ]:
# your code here

<H2>BONUS QUESTION - 15 points</H2>
Create a table called trade_blotter<br>
The columns will be trade date, the name of every security traded (Note: 10 columns for the security names)<br>
populate the table

In [ ]:
# your code here